In [19]:
# silly math
import mpmath as mpm
mpm.mp.prec=10
number = 12345.0987654
print(number % 1)
print(float(str(number)[str(number).index('.'):]))
print(mp.fmod(number, 1))


0.09876540000004752
0.0987654
0.099


https://en.wikipedia.org/wiki/File:Partial_transmittance.gif

Here are all the equations we will use in each function: 

Snell's Law:
* $nI*sin(thetaI) = nT*sin(thetaT)$
* $thetaT= arcsin((nI/nT)*sin(thetaI))$

Optical Path Length (OPL):
* $OPL = n*d$

There are two optical path lengths to consider:
* $opl1 = m*2*d*sin(thetaI)tan(thetaT)$ where $m$ is a integer between 0 and $\infty$
* $opl2 = m*2*d*(sec(thetaT))$ where $m$ is a integer between 0 and $\infty$

OPL diference:
* $oplD = oplM - opl1$ we want to compare all the optical path lengths with the first one that has one reflection. 

Phase from OPL or OPL difference:
* $phase = (2*pi*((OPL/wavelength)\%1)$
* $phase = 2*pi*arcsin(sin(2*pi*(OPL/wavelength)))$

Reflection and Transmission Coeffecents with snell's law to eliminate refractive index:
* $rPerp = -(sin(thetaI-thetaT))/(sin(thetaI+thetaT))$
* $rParr = +(tan(thetaI-thetaT))/(tan(thetaI+thetaT))$
* $tPerp = +(2*cos(thetaI)sin(thetaT))/(sin(thetaI+thetaT))$
* $tParr = +(2*cos(thetaI)sin(thetaT))/(sin(thetaI+thetaT)*cos(thetaI-thetaT))$

Electric field from reflection and transmission coefecents:
* $E_{0t} + E_{0r} = E_{0i}$
* $E_{0r}==$

Phase shift after reflection:
* $ $


add_two_phases_vectorally

In [24]:
import mpmath as mp

def thetaT_from_snell(theta1, n1, n2):
    theta2 = mp.asin((n1/n2)*mp.sin(theta1))
    return theta2

#thest two are from the geometry of the problem.
def opl_1(m, d, thetaI, thetaT):
    '''
    computes the optical path length outside of the slideglass
    '''
    opl1 = m*2*d*mp.sin(thetaI)*mp.tan(thetaT)
    return opl1

def opl_2(m, d, thetaI, thetaT):
    opl2 = m*2*d*(mp.sec(thetaT))
    return opl2

def generate_total_opl(numberOfRays, thetaI, n1, n2, d):
    thetaT = thetaT_from_snell(thetaI, n1, n2)
    opticalPathLengthList = []
    for m in range(numberOfRays):
        externalPath = opl_1((numberOfRays - 1 - m), d, thetaI, thetaT)
        internalPath = opl_2(m, d, thetaI, thetaT)
        oplTotal = externalPath + internalPath
        opticalPathLengthList.append(oplTotal)
    return(opticalPathLengthList, thetaT)

def calculate_opl_difference(numberOfRays, thetaI, n1, n2, d):
    '''
    make sure the first one is the shortest path we compare everything else to that one. 
    '''
    opticalPathLengthList = generate_total_opl(numberOfRays, thetaI, n1, n2, d)
    opticalPathLengthDifferenceList = []
    shortPath = opticalPathLengthList[0]
    for i, pathLength in enumerate(opticalPathLengthList):
        oplDiff = pathLength - shortPath
        opticalPathLengthDifferenceList.append(oplDiff)
    return(opticalPathLengthDifferenceList)

def fresnel_coeff(thetaI, thetaT):
    coeff = {}
    coeff['rPerp'] = -(mp.sin(thetaI-thetaT))/(mp.sin(thetaI+thetaT))
    coeff['rParr'] = +(mp.tan(thetaI-thetaT))/(mp.tan(thetaI+thetaT))
    coeff['tPerp'] = +(2*mp.cos(thetaI)*mp.sin(thetaT))/(mp.sin(thetaI+thetaT))
    coeff['tParr'] = +(2*mp.cos(thetaI)*mp.sin(thetaT))/(mp.sin(thetaI+thetaT)*mp.cos(thetaI-thetaT))
    return(coeff)

#polarization angle = polAng

def total_path_E_field_amplitude(numberOfRays, thetaI, thetaT, e0Perp = 1, e0Parr = 1):
    n1ToN2Coeff = fresnel_coeff(thetaI, thetaT)
    eFields = {'ePerp': [n1ToN2Coeff['rPerp']*e0Perp], 'eParr': [n1ToN2Coeff['rParr']*e0Parr]} # save the primary reflection
    '''
    r
    tRT
    tRRRT
    tRRRRRT
    ...etc
    '''
    n2ToN1Coeff = fresnel_coeff(thetaT, thetaI)
    for i in range(1, numberOfRays): # for any additional rays calculate the total E field
        numberOfInternalReflections = (i*2)-1
        ePerp = n1ToN2Coeff['tPerp'] * n2ToN1Coeff['rPerp']**numberOfInternalReflections * n2ToN1Coeff['tPerp'] * e0Perp
        eParr = n1ToN2Coeff['tParr'] * n2ToN1Coeff['rParr']**numberOfInternalReflections * n2ToN1Coeff['tParr'] * e0Parr
        eFields['ePerp'].append(ePerp)
        eFields['eParr'].append(eParr)
    return(eFields)

In [27]:
# example:
# Constants
numberOfRays = 5
n1 = 1.0
n2 = 1.45
wavelength = 650.0E-6
thetaI = mp.pi/4
d = 0.1E-3#mp.sqrt(2)/2.0
dtheta = 6.0E-3 # 6 milliradians (three on each side of 45 deg)

oplList, thetaT = generate_total_opl(numberOfRays, thetaI, n1, n2, d)
eFieldAmplitudes = total_path_E_field_amplitude(numberOfRays, thetaI, thetaT)



In [23]:
# Tests
# Constants
n1 = 1.0
n2 = 1.45
wavelength = 650.0E-6
thetaI = mp.pi/4
d = 0.1E-3#mp.sqrt(2)/2.0
dtheta = 6.0E-3 # 6 milliradians (three on each side of 45 deg)

thetaT = thetaT_from_snell(thetaI, n1, n2)
#print(fresnel_coeff(thetaI, thetaT))
#print(phase_shift_on_reflection(thetaI, thetaT))

print(total_path_E_field(5, thetaI, thetaT))

{'eParr': [mpf('0.080078'), mpf('-0.07959'), mpf('-0.00051022'), mpf('-3.2745e-6'), mpf('-2.1013e-8')], 'ePerp': [mpf('-0.28271'), mpf('0.26025'), mpf('0.020752'), mpf('0.0016613'), mpf('0.00013256')]}


#### For the case of three interfereing rays:

|         | external ray | Internal ray |
|---------|--------------|--------------|
| ray one | 0            | 2            |
| ray two | 1            | 1            |
|ray three| 2            | 0            |

In [4]:
hello = 0
for i in range(1, 10):
    hello += 3
    print(i)

1
2
3
4
5
6
7
8
9


In [3]:
for i in range(1, 2):
    print('hi')

hi


In [0]:
# graveyard:


def total_path_reflectance_and_transmitance(numberOfRays, thetaI, n1, n2):
    '''
    r
    tRT
    tRRRT
    tRRRRRT
    ...etc
    '''
    
    thetaT = thetaT_from_snell(thetaI, n1, n2)
    n1ToN2Coeff = fresnel_coeff(thetaI, thetaT)
    n2ToN1Coeff = fresnel_coeff(thetaT, thetaI)
    
    
    R1Perp = (n1ToN2Coeff['rPerp'])**2
    R1Parr = (n1ToN2Coeff['rParr'])**2
    T1Perp = 1 - R1Perp
    T1Parr = 1 - R1Parr
    
    R2Perp = (n2ToN1Coeff['rPerp'])**2
    R2Parr = (n2ToN1Coeff['rParr'])**2
    T2Perp = 1 - R2Perp
    T2Parr = 1 - R2Parr
    
    #intensityByPerpAndParr = {'Perp': [R1Perp], 'Parr':[R1Parr]}
    def average_silly(a, b): # DANGER THIS IS THE WRONG WAY TO DO THIS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        c = (a+b)/(2)
        return(c)
    
    refTransTot = [average_silly(R1Perp, R1Parr)]
    
    for i in range(1, numberOfRays):
        
        refTransPerp = T1Perp * R2Perp**i * T2Perp # product of the reflectance and transmittance
        refTransParr = T1Parr * R2Parr**i * T2Parr
        
        #intensityByPerpAndParr['Perp'].append(refTransPerp)
        #intensityByPerpAndParr['Parr'].append(refTransParr)
        refTransTot.append(average_silly(refTransPerp, refTransParr))
        
    #return intensityByPerpAndParr
    return refTransTot
    

def phase_shift_on_reflection(thetaI, thetaT):
    coeff = fresnel_coeff(thetaI, thetaT)
    angle = mp.atan2(coeff['rPerp'], coeff['rParr'])
    return angle*(180/mp.pi)